# Initialization

In [1]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('../input/twitterdataset2/big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))




#Naive Bayes
from nltk.corpus import words as wsss
import pandas as pd 
from nltk.tokenize import WordPunctTokenizer
import re
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.tree import DecisionTreeClassifier
from nltk.corpus import stopwords
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from nltk.classify import SklearnClassifier
from nltk.corpus import names
import numpy as np
import time
import matplotlib.pyplot as plt
from pandas import *

wordss=[]
for i in wsss.words():
    wordss.append(i)

start = time.time()
num=0
df = read_csv("../input/twitterdataset/traindata.csv")
df1 = df['tag']
df2 = df['content']
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

f= open("processed_dataset.csv","w")

def clean(s):
    def tweet_cleaner(text):
        soup = BeautifulSoup(text, 'lxml')
        souped = soup.get_text()
        stripped = re.sub(combined_pat, '', souped)
        try:
            clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
        except:
            clean = stripped
        letters_only = re.sub("[^a-zA-Z]", " ", clean)
        lower_case = letters_only.lower()
        words = tok.tokenize(lower_case)
        return (" ".join(words)).strip()
    
    stopWords = set(stopwords.words('english'))
    ps = PorterStemmer()
    li=[]
    test_result = []
    wordsFiltered = []
    b=tweet_cleaner(s)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(s))
    #print(words)
    
    wordsFiltered=[]
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)

    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    f.write(',')
    for i in tmp:
        f.write(i+' ')
    f.write('\n')
    
    #print(li)
    return tmp
    
    
    #return words


p=[]
s=[]
n=[]
c=[]
counterz = 0
for t in range(0,len(df2)):
    if counterz%100==0:
        print(counterz)
    counterz=counterz+1
    if df1[t]=='politics':
        sen=df2[t]
        f.write('0')
        p=p+clean(sen)
    elif df1[t]=='sports':
        sen=df2[t]
        f.write('1')
        s=s+clean(sen)
    elif df1[t]=='natural':
        sen=df2[t]
        f.write('2')
        n=n+clean(sen)
    elif df1[t]=='crime':
        sen=df2[t]
        f.write('3')
        c=c+clean(sen)
        
f.close()
        
#print(p)
#print(s)
#print(n)
#print(c)

normal_vocab=[]
def word_feats(words):
    return dict([(words, True)]) 


for a in p:
    if a in p and a in s and a in n and a in c:
        normal_vocab=normal_vocab+list(a);
        p.remove(a)
        s.remove(a)
        n.remove(a)
        c.remove(a)
    if a in p and a in s:
        normal_vocab=normal_vocab+list(a);
        p.remove(a)
        s.remove(a)
    if a in s and a in n:
        normal_vocab=normal_vocab+list(a);
        n.remove(a)
        s.remove(a)
    if a in n and a in c:
        normal_vocab=normal_vocab+list(a);
        n.remove(a)
        c.remove(a)
    if a in p and a in c:
        normal_vocab=normal_vocab+list(a);
        p.remove(a)
        c.remove(a)
    if a in s and a in c:
        normal_vocab=normal_vocab+list(a);
        c.remove(a)
        s.remove(a)
    if a in p and a in n:
        normal_vocab=normal_vocab+list(a);
        p.remove(a)
        n.remove(a)
    if a in p and a in s and a in n:
        normal_vocab=normal_vocab+list(a);
        p.remove(a)
        s.remove(a)
        n.remove(a)
    if a in s and a in n and a in c:
        normal_vocab=normal_vocab+list(a);
        c.remove(a)
        s.remove(a)
        n.remove(a)
    if a in p and a in n and a in c:
        normal_vocab=normal_vocab+list(a);
        p.remove(a)
        c.remove(a)
        n.remove(a)
    if a in p and a in s and a in c:
        normal_vocab=normal_vocab+list(a);
        p.remove(a)
        s.remove(a)
        c.remove(a)

        
politics_vocab=p
#print(politics_vocab)
sports_vocab=s
natural_vocab=n
crime_vocab=c
 
politics_features = [(word_feats(pol), 'pol') for pol in politics_vocab]
sports_features = [(word_feats(spo), 'spo') for spo in sports_vocab]
natural_features = [(word_feats(nat), 'nat') for nat in natural_vocab]
crime_features = [(word_feats(cri), 'cri') for cri in crime_vocab]
normal_features = [(word_feats(nor), 'nor') for nor in normal_vocab]
train_set = politics_features + sports_features + natural_features+crime_features
#print(train_set)


0
100
200
300
400
500
600
700


# Naive Bayes

In [2]:
classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
#classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/test.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 2, 0, 0, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 0, 0, 1, 2, 1, 0, 3, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 3, 0, 1, 2, 1, 2, 2, 0, 1, 3, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 1, 1, 1, 0, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 3, 2, 2, 0, 0, 3, 2, 1, 3, 0, 3, 1, 2, 3, 0, 2, 0, 1, 0, 3, 2, 2, 1, 2, 1, 2, 1, 3, 0, 0, 2, 1, 0, 1, 2, 1, 1, 2, 0, 1, 3, 3, 2, 0, 0, 0, 3, 1, 2, 3, 2, 3, 1, 1, 2, 0, 2, 3, 1, 2, 3, 3, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 0, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 2, 1, 1]
[2, 0, 1, 3, 1, 0, 2, 2, 0, 1, 1, 1, 2, 1, 0, 2, 2, 0, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 0, 0, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 3, 0, 1, 2, 1, 2, 2, 0, 1, 3, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 1, 1, 0, 1, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 1, 1, 2, 0, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 0, 2, 1

# SVM

In [3]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
#classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/testv2.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2v2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 0, 2, 0]
[2, 0, 1, 3, 1, 0, 2, 2, 0, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0

# SVM, tol=10e-5

In [4]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
classifier = SklearnClassifier(LinearSVC(tol=1e-5, max_iter=20000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
#classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/testv2.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2v2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 0, 2, 0]
[2, 0, 1, 3, 1, 0, 2, 2, 0, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0

# SVM, tol=10e-7

In [5]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
classifier = SklearnClassifier(LinearSVC(tol=1e-7, max_iter=20000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
#classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/testv2.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2v2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 0, 2, 0]
[2, 0, 1, 3, 1, 0, 2, 2, 0, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0

# SVM, max_iter=10000

In [6]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=10000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
#classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/testv2.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2v2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 0, 2, 0]
[2, 0, 1, 3, 1, 0, 2, 2, 0, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0

# SVM, max_iter=30000

In [7]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=30000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
#classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/testv2.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2v2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 0, 2, 0]
[2, 0, 1, 3, 1, 0, 2, 2, 0, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0

# SVM, no class weight

In [8]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
#classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/testv2.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2v2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 0, 2, 0]
[2, 0, 1, 3, 1, 0, 2, 2, 0, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 1, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0

# SVM, C=10

In [9]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, class_weight='balanced', C=10, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
#classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/testv2.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2v2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 0, 2, 0]
[2, 0, 1, 3, 1, 0, 2, 2, 0, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0

# SVM, C=1000

In [10]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, class_weight='balanced', C=1000, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
#classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/testv2.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2v2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 0, 2, 0]
[2, 0, 1, 3, 1, 0, 2, 2, 0, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0

# SVM, no loss

In [11]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, class_weight='balanced', C=100, multi_class='ovr', random_state=0), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
#classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/testv2.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2v2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 0, 2, 0]
[2, 0, 1, 3, 1, 0, 2, 2, 0, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0

# Logistic Regression

In [12]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
#classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/test.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 2, 0, 0, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 0, 0, 1, 2, 1, 0, 3, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 3, 0, 1, 2, 1, 2, 2, 0, 1, 3, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 1, 1, 1, 0, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 3, 2, 2, 0, 0, 3, 2, 1, 3, 0, 3, 1, 2, 3, 0, 2, 0, 1, 0, 3, 2, 2, 1, 2, 1, 2, 1, 3, 0, 0, 2, 1, 0, 1, 2, 1, 1, 2, 0, 1, 3, 3, 2, 0, 0, 0, 3, 1, 2, 3, 2, 3, 1, 1, 2, 0, 2, 3, 1, 2, 3, 3, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 0, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 2, 1, 1]
[2, 0, 1, 3, 1, 0, 2, 2, 0, 1, 1, 1, 2, 1, 0, 2, 2, 0, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 0, 0, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 3, 0, 1, 2, 1, 2, 2, 0, 1, 3, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 1, 1, 0, 1, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 1, 1, 2, 0, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 0, 2, 1

# Decision Tree

In [13]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
#classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/testv2.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2v2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 0, 2, 0]
[2, 0, 1, 3, 1, 0, 2, 2, 0, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0

# Random Forest

In [14]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/test.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 2, 0, 0, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 0, 0, 1, 2, 1, 0, 3, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 3, 0, 1, 2, 1, 2, 2, 0, 1, 3, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 1, 1, 1, 0, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 3, 2, 2, 0, 0, 3, 2, 1, 3, 0, 3, 1, 2, 3, 0, 2, 0, 1, 0, 3, 2, 2, 1, 2, 1, 2, 1, 3, 0, 0, 2, 1, 0, 1, 2, 1, 1, 2, 0, 1, 3, 3, 2, 0, 0, 0, 3, 1, 2, 3, 2, 3, 1, 1, 2, 0, 2, 3, 1, 2, 3, 3, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 0, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 2, 1, 1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

# Random Forest, n_estimator=3

In [15]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
classifier = SklearnClassifier(RandomForestClassifier(n_estimators=3, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/test.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 2, 0, 0, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 0, 0, 1, 2, 1, 0, 3, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 3, 0, 1, 2, 1, 2, 2, 0, 1, 3, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 1, 1, 1, 0, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 3, 2, 2, 0, 0, 3, 2, 1, 3, 0, 3, 1, 2, 3, 0, 2, 0, 1, 0, 3, 2, 2, 1, 2, 1, 2, 1, 3, 0, 0, 2, 1, 0, 1, 2, 1, 1, 2, 0, 1, 3, 3, 2, 0, 0, 0, 3, 1, 2, 3, 2, 3, 1, 1, 2, 0, 2, 3, 1, 2, 3, 3, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 0, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 2, 1, 1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

# Random Forest, n_estimators=7

In [16]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
classifier = SklearnClassifier(RandomForestClassifier(n_estimators=7, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/test.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 2, 0, 0, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 0, 0, 1, 2, 1, 0, 3, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 3, 0, 1, 2, 1, 2, 2, 0, 1, 3, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 1, 1, 1, 0, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 3, 2, 2, 0, 0, 3, 2, 1, 3, 0, 3, 1, 2, 3, 0, 2, 0, 1, 0, 3, 2, 2, 1, 2, 1, 2, 1, 3, 0, 0, 2, 1, 0, 1, 2, 1, 1, 2, 0, 1, 3, 3, 2, 0, 0, 0, 3, 1, 2, 3, 2, 3, 1, 1, 2, 0, 2, 3, 1, 2, 3, 3, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 0, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 2, 1, 1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

# Random Forest, max_depth=5

In [17]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=5, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/test.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 2, 0, 0, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 0, 0, 1, 2, 1, 0, 3, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 3, 0, 1, 2, 1, 2, 2, 0, 1, 3, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 1, 1, 1, 0, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 3, 2, 2, 0, 0, 3, 2, 1, 3, 0, 3, 1, 2, 3, 0, 2, 0, 1, 0, 3, 2, 2, 1, 2, 1, 2, 1, 3, 0, 0, 2, 1, 0, 1, 2, 1, 1, 2, 0, 1, 3, 3, 2, 0, 0, 0, 3, 1, 2, 3, 2, 3, 1, 1, 2, 0, 2, 3, 1, 2, 3, 3, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 0, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 2, 1, 1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

# Random Forest, max_depth=15

In [18]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=15, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/test.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 2, 0, 0, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 0, 0, 1, 2, 1, 0, 3, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 3, 0, 1, 2, 1, 2, 2, 0, 1, 3, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 1, 1, 1, 0, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 3, 2, 2, 0, 0, 3, 2, 1, 3, 0, 3, 1, 2, 3, 0, 2, 0, 1, 0, 3, 2, 2, 1, 2, 1, 2, 1, 3, 0, 0, 2, 1, 0, 1, 2, 1, 1, 2, 0, 1, 3, 3, 2, 0, 0, 0, 3, 1, 2, 3, 2, 3, 1, 1, 2, 0, 2, 3, 1, 2, 3, 3, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 0, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 2, 1, 1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

# KNN

In [19]:
#classifier = NaiveBayesClassifier.train(train_set)
##classifier = SklearnClassifier(LinearSVC(random_state=0), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
#classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/testv2.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2v2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    #print(words)
    
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print(wordsFiltered)
    for word in wordsFiltered:
        l=ps.stem(word)
        li.append(l)
    #print(li)
    
    tmp=[]
    for i in li:
        if i not in tmp:
            if i!='b':
                if len(i)==1:
                    continue
                elif len(i)<3:
                    if i in wordss:
                        tmp.append(correction(i))
                else:
                    tmp.append(correction(i))
    
    sentence = tmp
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 3, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 0, 2, 0]
[2, 0, 1, 3, 1, 0, 2, 0, 0, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 0, 1, 2, 3, 0, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0

# SVM [18]

In [20]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('../input/twitterdataset2/big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))




#Naive Bayes
from nltk.corpus import words as wsss
import pandas as pd 
from nltk.tokenize import WordPunctTokenizer
import re
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.tree import DecisionTreeClassifier
from nltk.corpus import stopwords
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from nltk.classify import SklearnClassifier
from nltk.corpus import names
import numpy as np
import time
import matplotlib.pyplot as plt
from pandas import *

wordss=[]
for i in wsss.words():
    wordss.append(i)

start = time.time()
num=0
df = read_csv("../input/twitterdataset/traindata.csv")
df1 = df['tag']
df2 = df['content']
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

f= open("processed_dataset.csv","w")

def clean(s):
    def tweet_cleaner(text):
        soup = BeautifulSoup(text, 'lxml')
        souped = soup.get_text()
        stripped = re.sub(combined_pat, '', souped)
        try:
            clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
        except:
            clean = stripped
        letters_only = re.sub("[^a-zA-Z]", " ", clean)
        lower_case = letters_only.lower()
        words = tok.tokenize(lower_case)
        return (" ".join(words)).strip()
    
    stopWords = set(stopwords.words('english'))
    ps = PorterStemmer()
    li=[]
    test_result = []
    wordsFiltered = []
    b=tweet_cleaner(s)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(s))
    #print(words)
    
    
    return words


p=[]
s=[]
n=[]
c=[]
counterz = 0
for t in range(0,len(df2)):
    counterz=counterz+1
    if df1[t]=='politics':
        sen=df2[t]
        f.write('0')
        p=p+clean(sen)
    elif df1[t]=='sports':
        sen=df2[t]
        f.write('1')
        s=s+clean(sen)
    elif df1[t]=='natural':
        sen=df2[t]
        f.write('2')
        n=n+clean(sen)
    elif df1[t]=='crime':
        sen=df2[t]
        f.write('3')
        c=c+clean(sen)
        
f.close()
        
#print(p)
#print(s)
#print(n)
#print(c)

normal_vocab=[]
def word_feats(words):
    return dict([(words, True)]) 


for a in p:
    if a in p and a in s and a in n and a in c:
        normal_vocab=normal_vocab+list(a);
        p.remove(a)
        s.remove(a)
        n.remove(a)
        c.remove(a)
    if a in p and a in s:
        normal_vocab=normal_vocab+list(a);
        p.remove(a)
        s.remove(a)
    if a in s and a in n:
        normal_vocab=normal_vocab+list(a);
        n.remove(a)
        s.remove(a)
    if a in n and a in c:
        normal_vocab=normal_vocab+list(a);
        n.remove(a)
        c.remove(a)
    if a in p and a in c:
        normal_vocab=normal_vocab+list(a);
        p.remove(a)
        c.remove(a)
    if a in s and a in c:
        normal_vocab=normal_vocab+list(a);
        c.remove(a)
        s.remove(a)
    if a in p and a in n:
        normal_vocab=normal_vocab+list(a);
        p.remove(a)
        n.remove(a)
    if a in p and a in s and a in n:
        normal_vocab=normal_vocab+list(a);
        p.remove(a)
        s.remove(a)
        n.remove(a)
    if a in s and a in n and a in c:
        normal_vocab=normal_vocab+list(a);
        c.remove(a)
        s.remove(a)
        n.remove(a)
    if a in p and a in n and a in c:
        normal_vocab=normal_vocab+list(a);
        p.remove(a)
        c.remove(a)
        n.remove(a)
    if a in p and a in s and a in c:
        normal_vocab=normal_vocab+list(a);
        p.remove(a)
        s.remove(a)
        c.remove(a)

        
politics_vocab=p
#print(politics_vocab)
sports_vocab=s
natural_vocab=n
crime_vocab=c
 
politics_features = [(word_feats(pol), 'pol') for pol in politics_vocab]
sports_features = [(word_feats(spo), 'spo') for spo in sports_vocab]
natural_features = [(word_feats(nat), 'nat') for nat in natural_vocab]
crime_features = [(word_feats(cri), 'cri') for cri in crime_vocab]
normal_features = [(word_feats(nor), 'nor') for nor in normal_vocab]
train_set = politics_features + sports_features + natural_features+crime_features
#print(train_set)

#classifier = NaiveBayesClassifier.train(train_set)
classifier = SklearnClassifier(LinearSVC(max_iter=20000), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LinearSVC(tol=1e-6, max_iter=20000, class_weight='balanced', C=100, multi_class='ovr', random_state=0, loss='hinge'), sparse=False).train(train_set) 
#classifier = SklearnClassifier(LogisticRegression(), sparse=False).train(train_set)
#classifier = SklearnClassifier(DecisionTreeClassifier(), sparse=False).train(train_set)
#classifier = SklearnClassifier(RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0, criterion="entropy"), sparse=False).train(train_set) 
#classifier = SklearnClassifier(KNeighborsClassifier(), sparse=False).train(train_set) 

cols = ['text']
dt = pd.read_csv("../input/twitterdataset/test.csv",header=None, names=cols)
dtm = read_csv("../input/twitterdataset/testmatch2.csv")
dtm1 = dtm['tag']
dtm2 = dtm['content']
#print(classifier)





# Predict
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


testing = dt.text[:]
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
li=[]
test_result = []
wordsFiltered = []

c=0
n=0
po=0
sp=0
cr=0
na=0
poli=0
spor=0
natu=0
crim=0
o=0

counterz=0

import numpy as np
actual = []
predicted = []

for t in testing:
    if counterz%20==0:
        print(counterz)
    counterz=counterz+1
    pol=0
    cri=0
    nat=0
    spo=0
    pol2=0
    cri2=0
    nat2=0
    spo2=0
    #print(t)
    
    #print(wd)
    b=tweet_cleaner(t)
    #print(b)
    wd = word_tokenize(b)
    test_result.append(b)
    words = word_tokenize(tweet_cleaner(t))
    
    sentence = words
    #print(sentence)
    #sentence = sentence.lower()
    #words = sentence.split(' ')
    w=sentence
    wordsFiltered=[]
    for word in w:
        classResult = classifier.classify( word_feats(word))
        if classResult == 'pol':
            pol = pol + 1
        if classResult == 'spo':
            spo = spo + 1
        if classResult == 'nat':
            nat = nat + 1
        if classResult == 'cri':
            cri = cri + 1
    #print(neg)
    #print(pos)
    #print(li)
    #print(len(words))

    pol2=float(pol)/(len(wd))
    spo2=float(spo)/(len(wd))
    nat2=float(nat)/(len(wd))
    cri2=float(cri)/(len(wd))

    #print('Politics: ' + str(pol2))
    #print('Sports: ' + str(spo2))
    #print('Natural: ' + str(nat2))
    #print('Crime: ' + str(cri2))
    
    
    if dtm1[o]=='politics':
        actual.append(0)
    elif dtm1[o]=='sports':
        actual.append(1)
    elif dtm1[o]=='natural':
        actual.append(2)
    elif dtm1[o]=='crime':
        actual.append(3)
    
    if pol2>=spo2 and pol2>=nat2 and pol2>=cri2:
        po=po+1
        predicted.append(0)
        if dtm1[o]=='politics':
            poli=poli+1
            #print('politics')
    elif spo2>=pol2 and spo2>=nat2 and spo2>=cri2:
        sp=sp+1
        predicted.append(1)
        if dtm1[o]=='sports':
            spor=spor+1
            #print('sports')
    elif nat2>=spo2 and nat2>=pol2 and nat2>=cri2:
        na=na+1
        predicted.append(2)
        if dtm1[o]=='natural':
            natu=natu+1
            #print('natural')
    elif cri2>=spo2 and cri2>=nat2 and cri2>=pol2:
        cr=cr+1
        predicted.append(3)
        if dtm1[o]=='crime':
            crim=crim+1
            #print('crime')
    
    #print(li)
    while len(li)>0:
        li.pop()
    
    c=c+float(pol2+spo2+nat2+cri2)
    n=n+1
    o=o+1



#np.save("actual.npy", np.array(actual))
#np.save("SVMpredicted.npy", np.array(predicted))

print(actual)
print(predicted)

politics_count=0
sports_count=0
natural_count=0
crime_count=0
for i in range(0, len(testing)):
    if dtm1[i]=='politics':
        politics_count=politics_count+1
    elif dtm1[i]=='sports':
        sports_count=sports_count+1
    elif dtm1[i]=='natural':
        natural_count=natural_count+1
    elif dtm1[i]=='crime':
        crime_count=crime_count+1
print('Politics Count: ' + str(politics_count))
print('Sports Count: ' + str(sports_count))
print('Natural Count: ' + str(natural_count))
print('Crime Count: ' + str(crime_count))
print()



ac=poli+spor+crim+natu
print('Politics Predicted: ' + str(po))
print('Sports Predicted: ' + str(sp))
print('Natural Predicted: ' + str(na))
print('Crime Predicted: ' + str(cr))

print()
print('Politics Correct: ' + str(poli))
print('Sports Correct: ' + str(spor))
print('Natural Correct: ' + str(natu))
print('Crime Correct: ' + str(crim))
print()

print('Test Data: ' + str(n))
print('Accuracy: ' + str(ac/n))
end = time.time()
print('Run Time: ' +str(end - start))# -*- coding: utf-8 -*-



0
20
40
60
80
100
120
140
160
180
[2, 0, 1, 3, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 2, 0, 0, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 0, 0, 1, 2, 1, 0, 3, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 3, 0, 1, 2, 1, 2, 2, 0, 1, 3, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 1, 1, 1, 0, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 3, 2, 2, 0, 0, 3, 2, 1, 3, 0, 3, 1, 2, 3, 0, 2, 0, 1, 0, 3, 2, 2, 1, 2, 1, 2, 1, 3, 0, 0, 2, 1, 0, 1, 2, 1, 1, 2, 0, 1, 3, 3, 2, 0, 0, 0, 3, 1, 2, 3, 2, 3, 1, 1, 2, 0, 2, 3, 1, 2, 3, 3, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 0, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 2, 1, 1]
[2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 2, 2, 0, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 0, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 1